In [8]:
from scapy.all import rdpcap, UDP
from scapy.packet import bind_layers, Packet
from scapy.fields import *

# Define a custom Matter layer
class Matter(Packet):
    name = "Matter"
    fields_desc = [StrLenField("data", "", length_from=lambda pkt: len(pkt.payload))]

# Bind UDP port 5540 (Matter) to the Matter layer
bind_layers(UDP, Matter, dport=5540)
bind_layers(UDP, Matter, sport=5540)

# Load the pcap file
packets = rdpcap('/home/user/matter_c1.pcap')

# Function to get the next layer name after UDP
def get_next_layer_name(packet):
    udp_payload = packet[UDP].payload
    next_layer_name = type(udp_payload).__name__
    return next_layer_name

# Extract the UDP payload from all frames whose UDP Payload is Matter. For specific packets (udp_payload = bytes(packets[289][UDP].payload))
matter_layer = []
for i, packet in enumerate(packets):
    if UDP in packet:
        udp_layer = packet[UDP]
        udp_payload_bytes = bytes(udp_layer.payload)
        # Check if UDP sport or dport is 5540 (Matter)
        if udp_layer.sport == 5540 or udp_layer.dport == 5540:
            next_layer_name = get_next_layer_name(packet)
            print(f'Processing frame {i + 1} of layer {next_layer_name} ...')
            matter_layer.append({
                'frame_number': i + 1,  # Frame numbers are typically 1-based
                'payload': udp_payload_bytes
            })

# print the Matter payloads
with open('matter_layer_hex.txt', 'w') as file:
    for matter_message in matter_layer:
        # Convert the payload to a hex string 
        hex_payload = matter_message['payload'].hex()

        # add spaces between each byte and add a newline every 16 bytes
        hex_payload = ' '.join(hex_payload[i:i+2] for i in range(0, len(hex_payload), 2))
        hex_payload = '\n'.join(hex_payload[i:i+47] for i in range(0, len(hex_payload), 48))
        
        payload_size = len(matter_message["payload"])
        total_size = payload_size + 8
        file.write(f'# Frame {matter_message["frame_number"]}, Payload Size: {payload_size} bytes (0x{payload_size:02x}), Total UDP Packet Size: 0x{total_size:02x}\n')        
        file.write(hex_payload)
        file.write(f'\n\n')
    # Save the hex string to a text file

Processing frame 597 of layer Matter ...
Processing frame 598 of layer Matter ...
Processing frame 603 of layer Matter ...
Processing frame 604 of layer Matter ...
Processing frame 605 of layer Matter ...
Processing frame 606 of layer Matter ...
Processing frame 607 of layer Matter ...
Processing frame 608 of layer Matter ...
Processing frame 609 of layer Matter ...
Processing frame 610 of layer Matter ...
Processing frame 611 of layer Matter ...
Processing frame 612 of layer Matter ...
Processing frame 613 of layer Matter ...
Processing frame 614 of layer Matter ...
Processing frame 615 of layer Matter ...
Processing frame 616 of layer Matter ...
Processing frame 617 of layer Matter ...
Processing frame 618 of layer Matter ...
Processing frame 619 of layer Matter ...
Processing frame 620 of layer Matter ...
Processing frame 621 of layer Matter ...
Processing frame 622 of layer Matter ...
Processing frame 623 of layer Matter ...
Processing frame 624 of layer Matter ...
Processing frame